In [1]:
# pip install shap=0.45.1

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score, mean_absolute_error
import xgboost as xgb
import matplotlib.pyplot as plt
import connectorx as cx
import math
import time
import shap
import seaborn as sns
import statistics
import mlflow
from mlflow.models import infer_signature
import gc
import os
#from explainerdashboard import RegressionExplainer, ExplainerDashboard

In [3]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',50)

In [4]:
# data = pd.read_csv('conn_vol_pred_FC_1_2_3_data.csv')

In [5]:
# data.info()

In [6]:
# data.head()

In [7]:
# data.columns

In [8]:
# final_places=set(data['cur_final_place'].unique().tolist()+data['next_final_place'].unique().tolist())
# places ={}
# for cnt,i in enumerate(final_places):
#     places.update({i:cnt})

# data['cur_final_place_encoded'] = list(map(lambda x:places[x],data['cur_final_place']))
# data['next_final_place_encoded'] = list(map(lambda x:places[x],data['next_final_place']))

In [9]:
# final_places=set(data['cur_areatype'].unique().tolist()+data['next_areatype'].unique().tolist())
# places ={}
# for cnt,i in enumerate(final_places):
#     places.update({i:cnt})

# data['cur_areatype_encoded'] = list(map(lambda x:places[x],data['cur_areatype']))
# data['next_areatype_encoded'] = list(map(lambda x:places[x],data['next_areatype']))

In [10]:
# input_feature_lst = ['highway_id', 'next_highway_id', 'region', 'next_region', 'roadwaytype_id',
#                      'next_roadwaytype_id', 'next_connected_count', 'next_pcc', 'cur_lanes_cat', 
#                      'next_lanes_cat', 'cur_spd_cat', 'next_spd_cat', 'cur_final_place_encoded', 
#                      'next_final_place_encoded', 'cur_areatype_encoded', 'next_areatype_encoded', 
#                      'cur_capacity', 'next_capacity', 'volume']

# target_feature_lst = ['next_volume']

In [11]:
# seen_df = data[data['next_volume']>0]
# unseen_df = data[data['next_volume']<0]

In [12]:
# uni_seen_df = seen_df[input_feature_lst+target_feature_lst].drop_duplicates()
# uni_seen_df.shape

In [13]:
# X,y = uni_seen_df[input_feature_lst],uni_seen_df[target_feature_lst]
# # Split the data into training and testing sets
# X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
# X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
# X_train.shape,X_test.shape,X_val.shape

In [14]:
X_test,y_test = pd.read_csv('conn_vol_pred_FC_1_2_3_X_test.csv'),pd.read_csv('conn_vol_pred_FC_1_2_3_y_test.csv')
X_test.shape,y_test.shape

((1925, 19), (1925, 1))

In [3]:
os.environ['MLFLOW_TRACKING_USERNAME'] = 'isgadmin'
os.environ['MLFLOW_TRACKING_PASSWORD'] = 'infosense12@'
tracking_server_uri = "https://ptt2-mlflow.isgsuite.com/"
mlflow.set_tracking_uri(tracking_server_uri)
tracking_uri = mlflow.get_tracking_uri()
print("Tracking URI:", tracking_uri)

Tracking URI: https://ptt2-mlflow.isgsuite.com/


In [5]:
experiment_name = "TurningMovement_240508"
run_name = "FC_1_2_3_lanes_speed_cat_with_capacity_1"
experiment = mlflow.get_experiment_by_name(experiment_name)

print(f"Experiment_id: {experiment.experiment_id}")
print(f"Experiment_Name: {experiment_name}")
print(f"Run Name: {run_name}")
print(f"Artifact Location: {experiment.artifact_location}")
print(f"Tags: {experiment.tags}")
print(f"Lifecycle_stage: {experiment.lifecycle_stage}")
print(f"Creation timestamp: {experiment.creation_time}")

Experiment_id: 50
Experiment_Name: TurningMovement_240508
Run Name: FC_1_2_3_lanes_speed_cat_with_capacity_1
Artifact Location: mlflow-artifacts:/50
Tags: {}
Lifecycle_stage: active
Creation timestamp: 1715161862412


In [7]:
all_runs

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.geh_error,metrics.r2score,metrics.geh_error_per,metrics.mae,...,tags.Model_Name,tags.Train Size,tags.No. Of Target Feature,tags.Test Size,tags.mlflow.runName,tags.mlflow.user,tags.mlflow.source.name,tags.target_feature,tags.input_feature_list,tags.mlflow.parentRunId
0,3026a93a3cf64a928d8915db7975f95d,50,FINISHED,mlflow-artifacts:/50/3026a93a3cf64a928d8915db7...,2024-05-20 07:10:17.287000+00:00,2024-05-20 07:10:33.286000+00:00,58.252,0.718,58.252,515.754,...,XGBoost,2882,1,618,FC_1_2_3 and next_volume>1500,isgadmin,/usr/local/lib/python3.11/site-packages/ipyker...,None,None,None
1,94cdf09be2d04f898491d9d2cd834e0b,50,FINISHED,mlflow-artifacts:/50/94cdf09be2d04f898491d9d2c...,2024-05-15 16:20:22.455000+00:00,2024-05-15 16:24:03.927000+00:00,58.597,0.897,58.597,316.740,...,XGBoost,8979,1,1925,FC_1_2_3_srqt_transformation_1,isgadmin,C:\Users\AshishPrajapati\anaconda3\Lib\site-pa...,None,None,None
2,73b717709cae4437985d098cd6f7bb8a,50,FINISHED,mlflow-artifacts:/50/73b717709cae4437985d098cd...,2024-05-15 13:46:24.216000+00:00,2024-05-15 13:48:18.368000+00:00,59.273,0.895,59.273,320.482,...,XGBoost,8979,1,1925,FC_1_2_3_srqt_transformation,isgadmin,C:\Users\AshishPrajapati\anaconda3\Lib\site-pa...,None,None,None
3,bae57d8d4d5944bd863fe52eeaa9faad,50,FINISHED,mlflow-artifacts:/50/bae57d8d4d5944bd863fe52ee...,2024-05-15 12:25:57.992000+00:00,2024-05-15 12:27:26.481000+00:00,58.961,0.898,58.961,318.834,...,XGBoost,8979,1,1925,FC_1_2_3_group_roadwaytype_id_1,isgadmin,C:\Users\AshishPrajapati\anaconda3\Lib\site-pa...,None,None,None
4,f83b9b7061bd4244b9b3e85f96f5a84d,50,FINISHED,mlflow-artifacts:/50/f83b9b7061bd4244b9b3e85f9...,2024-05-15 12:19:20.026000+00:00,2024-05-15 12:21:47.560000+00:00,58.234,0.896,58.234,318.284,...,XGBoost,8979,1,1925,FC_1_2_3_group_roadwaytype_id,isgadmin,C:\Users\AshishPrajapati\anaconda3\Lib\site-pa...,None,None,None
5,a3c1b038dc074008b53ed794ded034c8,50,FINISHED,mlflow-artifacts:/50/a3c1b038dc074008b53ed794d...,2024-05-15 12:05:19.169000+00:00,2024-05-15 12:07:32.452000+00:00,60.104,0.893,60.104,324.905,...,XGBoost,8981,1,1925,FC_1_2_3_group_highway_id_1,isgadmin,C:\Users\AshishPrajapati\anaconda3\Lib\site-pa...,None,None,None
6,bb26eadd4cb643c1b3f261b23eacd6a0,50,FINISHED,mlflow-artifacts:/50/bb26eadd4cb643c1b3f261b23...,2024-05-15 12:00:03.370000+00:00,2024-05-15 12:02:55.320000+00:00,60.312,0.893,60.312,325.183,...,XGBoost,8981,1,1925,FC_1_2_3_group_highway_id,isgadmin,C:\Users\AshishPrajapati\anaconda3\Lib\site-pa...,None,None,None
7,d183e05d2c004147ad479c077a94e3b4,50,FINISHED,mlflow-artifacts:/50/d183e05d2c004147ad479c077...,2024-05-15 10:49:45.820000+00:00,2024-05-15 10:52:48.949000+00:00,60.468,0.892,60.468,325.901,...,XGBoost,8981,1,1925,FC_1_2_3_with_mae_as_eval_matrix_2,isgadmin,C:\Users\AshishPrajapati\anaconda3\Lib\site-pa...,None,None,None
8,746b98f6e832434f82befc44a37d5cc6,50,FINISHED,mlflow-artifacts:/50/746b98f6e832434f82befc44a...,2024-05-15 10:36:05.913000+00:00,2024-05-15 10:38:13.614000+00:00,60.416,0.894,60.416,324.218,...,XGBoost,8981,1,1925,FC_1_2_3_with_mae_as_eval_matrix_1,isgadmin,C:\Users\AshishPrajapati\anaconda3\Lib\site-pa...,None,None,None
9,52f471fb6d1b412089340e0ede8a6093,50,FINISHED,mlflow-artifacts:/50/52f471fb6d1b412089340e0ed...,2024-05-15 10:08:51.625000+00:00,2024-05-15 10:12:31.883000+00:00,60.156,0.888,60.156,328.429,...,XGBoost,8981,1,1925,FC_1_2_3_with_mae_as_eval_matrix,isgadmin,C:\Users\AshishPrajapati\anaconda3\Lib\site-pa...,None,None,None


In [6]:
all_runs = mlflow.search_runs(search_all_experiments=True,experiment_names=[experiment_name])
all_runs = mlflow.search_runs(search_all_experiments=True,experiment_names=[experiment_name])
Search_df = all_runs[['tags.mlflow.runName','run_id','artifact_uri','tags.input_feature_lst','tags.target_feature_lst']]

In [18]:
run_id = '8ebd2db8e9144178a158f80b6891de16'

In [19]:
# Search_df[Search_df['run_id'] == '8ebd2db8e9144178a158f80b6891de16']

In [20]:
# input_feature_lst = eval(Search_df[Search_df['tags.mlflow.runName'] == run_name]['tags.input_feature_lst'].values.tolist()[0])
# target_feature_lst = eval(Search_df[Search_df['tags.mlflow.runName'] == run_name]['tags.target_feature_lst'].values.tolist()[0])
# artifact_uri = Search_df[Search_df['tags.mlflow.runName'] == run_name]['artifact_uri'].values.tolist()[0]
# run_id = Search_df[Search_df['tags.mlflow.runName'] == run_name]['run_id'].values.tolist()[0]


In [21]:
input_feature_lst = eval(Search_df[Search_df['run_id'] == run_id]['tags.input_feature_lst'].values.tolist()[0])
target_feature_lst = eval(Search_df[Search_df['run_id'] == run_id]['tags.target_feature_lst'].values.tolist()[0])
artifact_uri = Search_df[Search_df['run_id'] == run_id]['artifact_uri'].values.tolist()[0]
run_id = Search_df[Search_df['run_id'] == run_id]['run_id'].values.tolist()[0]


In [22]:
input_feature_lst,target_feature_lst,artifact_uri,run_id

(['highway_id',
  'next_highway_id',
  'region',
  'next_region',
  'roadwaytype_id',
  'next_roadwaytype_id',
  'next_connected_count',
  'next_pcc',
  'cur_lanes_cat',
  'next_lanes_cat',
  'cur_spd_cat',
  'next_spd_cat',
  'cur_final_place_encoded',
  'next_final_place_encoded',
  'cur_areatype_encoded',
  'next_areatype_encoded',
  'cur_capacity',
  'next_capacity',
  'volume'],
 ['next_volume'],
 'mlflow-artifacts:/50/8ebd2db8e9144178a158f80b6891de16/artifacts',
 '8ebd2db8e9144178a158f80b6891de16')

In [23]:
logged_model = f'runs:/{run_id}/model'
# Load model as a PyFuncModel.
# loaded_model = mlflow.load_model(logged_model)
loaded_model = mlflow.sklearn.load_model(logged_model)

In [24]:
y_pred = loaded_model.predict(X_test)

In [25]:
mse = round(mean_squared_error(y_test, y_pred),3)
rmse = round(np.sqrt(mse),3)
r2 = round(r2_score(y_test, y_pred),3)
n = X_test.shape[0]
p = X_test.shape[1]
adjusted_r2score =  round((1 - (1 - r2) * (n - 1) / (n - p - 1)),3)
mae = round(mean_absolute_error(y_test, y_pred),3)
metrics = {'mse':mse,'rmse':rmse,'r2score':r2,'mae':mae,'adjusted_r2score' : adjusted_r2score}
metrics

{'mse': 257896.679,
 'rmse': 507.835,
 'r2score': 0.898,
 'mae': 313.866,
 'adjusted_r2score': 0.897}

In [26]:
explainer = RegressionExplainer(loaded_model, X_test, y_test,shap_kwargs=dict(check_additivity=False))

Changing class type to XGBRegressionExplainer...
Generating self.shap_explainer = shap.TreeExplainer(model)


In [27]:
ExplainerDashboard(explainer,decision_trees=False).run()

Building ExplainerDashboard..
Detected notebook environment, consider setting mode='external', mode='inline' or mode='jupyterlab' to keep the notebook interactive while the dashboard is running...
Generating layout...
Calculating shap values...
Calculating predictions...
Calculating residuals...
Calculating absolute residuals...
Calculating shap interaction values...
Reminder: TreeShap computational complexity is O(TLD^2), where T is the number of trees, L is the maximum number of leaves in any tree and D the maximal depth of any tree. So reducing these will speed up the calculation.
Calculating dependencies...
Calculating importances...
Reminder: you can store the explainer (including calculated dependencies) with explainer.dump('explainer.joblib') and reload with e.g. ClassifierExplainer.from_file('explainer.joblib')
Registering callbacks...
Starting ExplainerDashboard on http://172.30.26.122:8050


ConnectionError: HTTPConnectionPool(host='0.0.0.0', port=8050): Max retries exceeded with url: /_alive_1d00931f-d2da-4ed7-8493-aecf3816e7ca (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001689EDD5130>: Failed to establish a new connection: [WinError 10049] The requested address is not valid in its context'))

In [ ]:
# !pip install --user shap==0.45.1

In [ ]:
# !pip list

In [28]:
# shap == 0.45.1 server

# shap == 0.45.0 local